import the required libraries

In [9]:
import pandas as pd
import re
from collections import Counter
import sqlite3

Load the Wikipedia dataset ("wikipedia_dataset.xlsx") into a Pandas dataframe anad view it

In [10]:
# 1. Download the Wikipedia data
# 2. Take a look at the first 10 entries

Wikipedia_2016 = pd.read_excel("wikipedia_dataset.xlsx")
display(Wikipedia_2016.head(10))

,Page,2016-01-01 00:00:00,2016-01-02 00:00:00,2016-01-03 00:00:00,2016-01-04 00:00:00,2016-01-05 00:00:00,2016-01-06 00:00:00,2016-01-07 00:00:00,2016-01-08 00:00:00,2016-01-09 00:00:00,...,2016-12-22 00:00:00,2016-12-23 00:00:00,2016-12-24 00:00:00,2016-12-25 00:00:00,2016-12-26 00:00:00,2016-12-27 00:00:00,2016-12-28 00:00:00,2016-12-29 00:00:00,2016-12-30 00:00:00,2016-12-31 00:00:00
0,.xxx_en.wikipedia.org_all-access_all-agents,7089,7592,7436,7032,6498,6887,7209,7022,7194,...,4931.0,4034.0,5204.0,4331.0,4572.0,4509.0,5392.0,4950.0,2931.0,4101.0
1,.xxx_en.wikipedia.org_mobile-web_all-agents,6182,6447,6437,5912,5205,5770,5998,5948,6165,...,4617.0,3663.0,4882.0,4028.0,4263.0,4176.0,5068.0,4539.0,2666.0,3754.0
2,1._Juli_de.wikipedia.org_desktop_all-agents,20,49,33,49,34,26,66,28,18,...,26.0,20.0,19.0,43.0,24.0,41.0,39.0,39.0,31.0,31.0
3,2014_FIFA_World_Cup_en.wikipedia.org_all-acces...,5363,6280,6297,6288,6052,6200,5935,5958,6200,...,3356.0,3366.0,2912.0,2850.0,3140.0,3379.0,3497.0,3338.0,3329.0,3550.0
4,2014_FIFA_World_Cup_en.wikipedia.org_desktop_a...,3186,3784,3832,4059,4022,4213,3985,3823,3794,...,1719.0,1593.0,1255.0,1160.0,1311.0,1468.0,1728.0,1494.0,1515.0,1464.0
5,2014_FIFA_World_Cup_en.wikipedia.org_mobile-we...,2136,2425,2403,2171,1968,1930,1915,2071,2344,...,1603.0,1739.0,1623.0,1648.0,1797.0,1870.0,1722.0,1808.0,1783.0,2029.0
6,2015_Copa_AmÃ©rica_en.wikipedia.org_all-access...,1160,1197,838,924,878,995,940,1098,1203,...,388.0,403.0,329.0,321.0,424.0,408.0,428.0,399.0,426.0,361.0
7,2015_Copa_AmÃ©rica_en.wikipedia.org_desktop_al...,564,695,443,578,530,596,573,658,607,...,230.0,202.0,167.0,168.0,184.0,213.0,228.0,193.0,180.0,131.0
8,2015_Copa_AmÃ©rica_en.wikipedia.org_mobile-web...,580,485,390,336,338,395,357,425,575,...,157.0,200.0,157.0,152.0,234.0,188.0,197.0,203.0,243.0,228.0
9,2015_in_film_en.wikipedia.org_all-access_all-a...,54347,54784,55898,52521,42647,41312,40648,36065,38110,...,4006.0,4022.0,3929.0,4117.0,4464.0,4930.0,4697.0,4809.0,4631.0,4511.0


Print out some basic information about the dataframe

In [11]:
print(Wikipedia_2016.info(verbose = True))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 367 columns):
 #    Column               Dtype  
---   ------               -----  
 0    Page                 object 
 1    2016-01-01 00:00:00  int64  
 2    2016-01-02 00:00:00  int64  
 3    2016-01-03 00:00:00  int64  
 4    2016-01-04 00:00:00  int64  
 5    2016-01-05 00:00:00  int64  
 6    2016-01-06 00:00:00  int64  
 7    2016-01-07 00:00:00  int64  
 8    2016-01-08 00:00:00  int64  
 9    2016-01-09 00:00:00  int64  
 10   2016-01-10 00:00:00  int64  
 11   2016-01-11 00:00:00  int64  
 12   2016-01-12 00:00:00  int64  
 13   2016-01-13 00:00:00  int64  
 14   2016-01-14 00:00:00  int64  
 15   2016-01-15 00:00:00  int64  
 16   2016-01-16 00:00:00  int64  
 17   2016-01-17 00:00:00  int64  
 18   2016-01-18 00:00:00  int64  
 19   2016-01-19 00:00:00  int64  
 20   2016-01-20 00:00:00  int64  
 21   2016-01-21 00:00:00  int64  
 22   2016-01-22 00:00:00  int64  
 23   2016-01-23 

Our table has 1500 rows and 367 columns. To make working with the data easier, it will be best to reshape it so htat there are fewer columns.

In [12]:
# using pandas .melt(), unpivot the Wikipedia_2016 dataframe

Melted_Wikipedia = pd.melt(Wikipedia_2016, id_vars= "Page", var_name= "Date", value_name= "Visits")
print(Melted_Wikipedia.head())

                                                Page       Date  Visits
0        .xxx_en.wikipedia.org_all-access_all-agents 2016-01-01  7089.0
1        .xxx_en.wikipedia.org_mobile-web_all-agents 2016-01-01  6182.0
2        1._Juli_de.wikipedia.org_desktop_all-agents 2016-01-01    20.0
3  2014_FIFA_World_Cup_en.wikipedia.org_all-acces... 2016-01-01  5363.0
4  2014_FIFA_World_Cup_en.wikipedia.org_desktop_a... 2016-01-01  3186.0


Check the shape of our melted dataframe

In [13]:
Melted_Wikipedia.shape

(549000, 3)

At this point, we would want to know if our dataset contains any duplicate rows or missing values

In [14]:
# Check for contains duplicate entries

Melted_Wikipedia.duplicated().sum()

0

In [15]:
# Check for missing values
Melted_Wikipedia.isnull().sum()

Page       0
Date       0
Visits    87
dtype: int64

The next few steps would involve some feature engineering that is necessary to enable us answer the given questions
1. Add a column that tells us what day of the week a webpage was visited
2. A column with the information on the language a page was written in
3. What device was used in a search

In [17]:
# 1. Create a new column "Day" whose value is the day of the week a search occured

Melted_Wikipedia['Day'] = pd.to_datetime(Melted_Wikipedia['Date']).dt.day_name()
Melted_Wikipedia.head()

,Page,Date,Visits,Day
0,.xxx_en.wikipedia.org_all-access_all-agents,2016-01-01,7089.0,Friday
1,.xxx_en.wikipedia.org_mobile-web_all-agents,2016-01-01,6182.0,Friday
2,1._Juli_de.wikipedia.org_desktop_all-agents,2016-01-01,20.0,Friday
3,2014_FIFA_World_Cup_en.wikipedia.org_all-acces...,2016-01-01,5363.0,Friday
4,2014_FIFA_World_Cup_en.wikipedia.org_desktop_a...,2016-01-01,3186.0,Friday


A column for languages;
* One may infer which language a page is written in based on the page name
e.g. Special:Search_fr.wikipedia.org_all-access_all-agents is written in French and Special:Book_en.wikipedia.org_all-access_spider is written in English.

In [19]:
# 2a. Using .search() method of the Regex module, define a function that returns the 2 letters that represent the langauge of the page (eg., en for English), or "na" if language is not found
# 2b. Use .map() to call the defined function on the 'Page' column of the dataframe
# 2b. Using Counter, get the counts for each language

def get_language(page):
    res = re.search('[a-z][a-z].wikipedia.org',page)
    if res:
        return res[0][0:2]
    return 'na'

Melted_Wikipedia['Language'] = Melted_Wikipedia.Page.map(get_language)


print(Counter(Melted_Wikipedia.Language))

Counter({'en': 418338, 'ja': 44286, 'es': 28914, 'de': 14274, 'ru': 13176, 'fr': 12444, 'zh': 11346, 'na': 6222})


A column for devices;
For some of the pages in the dataset, it is possible to distinguish whether the visits to the page came from a desktop or a mobile device.
For example, consider Barack Obama's wikipedia page:
* Barack_Obama_en.wikipedia.org_desktop_all-agents : these visits came from desktop devices.
* Barack_Obama_en.wikipedia.org_mobile-web_all-agents: these visits came from mobile devices.

In [20]:
# 3a. Define a function using re to get the name of the device used to visit a web page
# 3b. Create the new column "device"
# 3c. Count the number of searches from each device

def get_device(page):
    dev = re.search('all-access|mobile-web|desktop',page)
    if dev:
        return dev[0][0:10]
    return 'na'

Melted_Wikipedia['Device'] = Melted_Wikipedia.Page.map(get_device)


print(Counter(Melted_Wikipedia.Device))

Counter({'all-access': 342210, 'mobile-web': 107604, 'desktop': 99186})


In [21]:
# Drop all missing values
Melted_Wikipedia.dropna(how= 'any', inplace = True)


In [22]:
# Take a look at the resulting dataframe
display(Melted_Wikipedia)

,Page,Date,Visits,Day,Language,Device
0,.xxx_en.wikipedia.org_all-access_all-agents,2016-01-01,7089.0,Friday,en,all-access
1,.xxx_en.wikipedia.org_mobile-web_all-agents,2016-01-01,6182.0,Friday,en,mobile-web
2,1._Juli_de.wikipedia.org_desktop_all-agents,2016-01-01,20.0,Friday,de,desktop
3,2014_FIFA_World_Cup_en.wikipedia.org_all-acces...,2016-01-01,5363.0,Friday,en,all-access
4,2014_FIFA_World_Cup_en.wikipedia.org_desktop_a...,2016-01-01,3186.0,Friday,en,desktop
...,...,...,...,...,...,...
548995,Zac_Efron_en.wikipedia.org_all-access_all-agents,2016-12-31,13146.0,Saturday,en,all-access
548996,Zayn_Malik_en.wikipedia.org_all-access_all-agents,2016-12-31,12597.0,Saturday,en,all-access
548997,Zendaya_en.wikipedia.org_all-access_all-agents,2016-12-31,6217.0,Saturday,en,all-access
548998,Zendaya_en.wikipedia.org_mobile-web_all-agents,2016-12-31,4650.0,Saturday,en,mobile-web


Convert the dataframe into an excel spreadsheet

In [23]:
# Use pd.to_excel()

Melted_Wikipedia.to_excel('Final_Wikipedia.xlsx')

Load the data into a SQL database
* sqlite3 has already been imported

In [24]:
# Read the data from the excel spreadsheet into a pandas dataframe

Wiki_sql = pd.read_excel('Final_Wikipedia.xlsx', header=0, index_col=0)

Create a new SQLite database connection object and assign it to a variable

In [25]:

db_connection = sqlite3.connect("wikipedia.db")

Establish a cursor object

In [26]:
cursor = db_connection.cursor()

Create the database tables:
Run c.execute() that takes in the SQL code that creates the database table as a string

In [ ]:
cursor.execute(
    """
    CREATE TABLE Wikipedia(
        date INTEGER,
        page TEXT NOT NULL,
        visits INTEGER,
        day TEXT NOT NULL,
        language TEXT NOT NULL,
        device TEXT NOT NULL
    );
"""
)


Populate the database

In [28]:
Wiki_sql.to_sql('Wikipedia',db_connection, if_exists='append', index=False)

You can query the database using .read_sql() to see

In [29]:
# Use pd.read_sql to show the first 10 rows from Wikepedia
pd.read_sql('SELECT * FROM Wikipedia LIMIT 10', db_connection)

,date,page,visits,day,language,device
0,2016-01-01 00:00:00,.xxx_en.wikipedia.org_all-access_all-agents,7089,Friday,en,all-access
1,2016-01-01 00:00:00,.xxx_en.wikipedia.org_mobile-web_all-agents,6182,Friday,en,mobile-web
2,2016-01-01 00:00:00,1._Juli_de.wikipedia.org_desktop_all-agents,20,Friday,de,desktop
3,2016-01-01 00:00:00,2014_FIFA_World_Cup_en.wikipedia.org_all-acces...,5363,Friday,en,all-access
4,2016-01-01 00:00:00,2014_FIFA_World_Cup_en.wikipedia.org_desktop_a...,3186,Friday,en,desktop
5,2016-01-01 00:00:00,2014_FIFA_World_Cup_en.wikipedia.org_mobile-we...,2136,Friday,en,mobile-web
6,2016-01-01 00:00:00,2015_Copa_AmÃ©rica_en.wikipedia.org_all-access...,1160,Friday,en,all-access
7,2016-01-01 00:00:00,2015_Copa_AmÃ©rica_en.wikipedia.org_desktop_al...,564,Friday,en,desktop
8,2016-01-01 00:00:00,2015_Copa_AmÃ©rica_en.wikipedia.org_mobile-web...,580,Friday,en,mobile-web
9,2016-01-01 00:00:00,2015_in_film_en.wikipedia.org_all-access_all-a...,54347,Friday,en,all-access
